In [1]:
from __future__ import print_function, division
import time, os
import numpy as np
import matplotlib.pyplot as plt
import sys
import networkx as nx
import pandas
from math import lgamma

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [2]:
repairs = pandas.read_csv('/home/cs231n/data/repairs.csv')

In [7]:
ex_repairs.columns

Index(['Dealer Cd', 'Chassis Division', 'Chassis\nReference\nNumber',
       'Plant Name', 'Model Vehicle', 'Build_Dt', 'Dlvry_Dt',
       'In Service Date', 'Miles', 'Rpr_Dt', 'ATA3', 'ATA3Desc', 'ATA6',
       'ATA6Desc', 'ATA9', 'ATA9Desc', 'Fail Type', 'Repair Cost', 'Dlr Story',
       'Chassis Build Month', 'Engine Build Month', 'Engine Manufacture Date',
       'Engine Displacement Liter Quantity', 'Engine Horsepower Number'],
      dtype='object')

In [3]:
ex_w = pandas.read_csv('/home/cs231n/data/Field_Snaps_With_warranty.txt')

In [23]:
print(min(ex_repairs['Chassis\nReference\nNumber'].unique()), max(ex_repairs['Chassis\nReference\nNumber'].unique()))

1.0 853.0


In [21]:
print(min(ex_nw['Veh Ref ID'].unique()), max(ex_nw['Veh Ref ID'].unique()))

814.0 1300.0


In [22]:
print(min(ex_w['Veh Ref ID'].unique()), max(ex_w['Veh Ref ID'].unique()))

1.0 813.0


In [4]:
selected_repairs = repairs[[
 'Chassis\nReference\nNumber',
 'Model Vehicle',
 'Build_Dt',
 'Dlvry_Dt',
 'In Service Date',
 'Miles',
 'Rpr_Dt',
 'ATA3',
 'ATA3Desc',
 'ATA6',
 'ATA6Desc',
 'ATA9',
 'ATA9Desc',
 'Fail Type',
 'Repair Cost']]
selected_repairs = selected_repairs[selected_repairs['Chassis\nReference\nNumber'].notnull()]

In [6]:
selected_repairs.iloc[850].values

array([628.0, 'T680   ', '6/24/2016', '7/27/2016', '7/27/2016', 406,
       '7/26/2016', 45, 'POWER PLANT (045)', 45021,
       'ELECTRONIC ENGINE CONTROL', 45021003,
       'ELECTRONIC CONTROL MODULE (ECU)', 'REFLASH ECU/ICU', 'very low'], dtype=object)

In [7]:
len(ex_w)

1581892

In [13]:
selected_w = ex_w[['Veh Ref ID',
 'Event DateTime',
 'Event Type Description',
 'Acc Pedal Position',
 'Ambient Air Temp',
 'Barometric Press',
 'Brake Switch',
 'Bus Utilization',
 'Cat Intake Gas Temp',
 'Cat Outlet Gas Temp',
 'Clutch Switch',
 'Cmd Eng Fuel Press',
 'Cruise Status',
 'Dpf Regen Inhibit Sw',
 'Dpf Thermal Mngmnt',
 'Drvr Demand Torque',
 'Eng Air Flow Rate',
 'Eng Avg Fuel Econ',
 'Eng Coolant Level',
 'Eng Coolant Temp',
 'Eng Demand Torque',
 'Eng DPF Intake Press',
 'Eng Egr Valve Pos',
 'Eng Exhaust Gas Temp',
 'Eng Fuel Del Press',
 'EngFuelTemp1',
 'Engine Speed',
 'Eng Man Abs Pressure',
 'Eng Oil Pressure',
 'EngInjRail1Press',
 'EngIntakeMan1Temp',
 'EngOilTemp1',
 'Eng Percent Torque',
 'EngTurbo1Boost',
 'EngTurbo1Pos',
 'EngTurbo1Speed',
 'Event - All Lamps On Time Hr',
 'Event - Amber Lamp Time Hr',
 'Event - Mil Lamp Time Hr',
 'Event - Red Lamp Time Hr',
 'Exhaust Tank Level',
 'Exhaust Tank Temp',
 'Fan Speed',
 'Keyswitch Bat Pot',
 'Part Trap Diff Press',
 'Part Trap Out Temp',
 'Scr Intake Gas Temp',
 'Scr Outlet Gas Temp',
 'Vehicle Speed',
 'Population',
 'DTCID',
 'Trip Distance',
 'Trip Idle Time',
 'Trip Run Time',
 'Altitude',
 'Engine Start Ambient',
 'Engine Start Coolant',
 'Ignition Cycle Counter',
 'Latitude',
 'Longitude',
 'Lifetime Idle Hours',
 'Lifetime Idle Fuel',
 'Lifetime Fuel',
 'Lifetime Distance',
 'Lifetime Engine Hours']]

In [14]:
ex_w_indexed = selected_w[:100000].set_index('Veh Ref ID')

In [15]:
repairs_indexed = selected_repairs.set_index('Chassis\nReference\nNumber')

In [16]:
joined_repairs_snapshots = ex_w_indexed.join(repairs_indexed)

In [188]:
high_repairs = selected_repairs[selected_repairs['Repair Cost'].isin(['high', 'very high'])]

In [189]:
veh_ids = high_repairs['Chassis\nReference\nNumber'].unique()

In [190]:
veh_ids

array([ 616.,   27.,   25.,  724.,  555.,  249.,  495.,  180.,  751.,
        801.,  373.,  340.,   40.])

In [89]:
veh_snapshots = selected_w[selected_w['Veh Ref ID'] == 797].sort_values(by='Event DateTime')

In [52]:
v0 = veh_snapshots.iloc[0]

In [90]:
veh_repairs = selected_repairs[selected_repairs['Chassis\nReference\nNumber'] == 797]

In [91]:
veh_repairs

,Chassis Reference Number,Model Vehicle,Build_Dt,Dlvry_Dt,In Service Date,Miles,Rpr_Dt,ATA3,ATA3Desc,ATA6,ATA6Desc,ATA9,ATA9Desc,Fail Type,Repair Cost
612,797.0,T680,1/19/2016,2/9/2016,2/9/2016,38236,6/23/2016,45,POWER PLANT (045),45021,ELECTRONIC ENGINE CONTROL,45021003,ELECTRONIC CONTROL MODULE (ECU),CALIBRATE WRONG,very low
652,797.0,W900B,3/16/2016,5/4/2016,5/4/2016,4735,6/13/2016,101,MX DIAG FAULT CODES (101),101001,MX DIAG FAULT CODES (001),101001001,PACCAR MX DIAGNOSTIC CODES,REFLASH ECU/ICU,very low
836,797.0,567,5/11/2016,6/22/2016,6/22/2016,529,5/27/2016,44,FUEL SYSTEM (044),44004,INJECTORS (004),44004001,INJECTOR ASSEMBLY,WEAK,medium
837,797.0,T680,2/5/2016,4/11/2016,4/11/2016,38030,7/18/2016,43,EXHAUST SYSTEM (043),43006,EMISSIONS (006),43006007,"SENSOR - DIFFERENTIAL PRESSURE, PARTICULATE TR...",CHECK CONDITION,very low


In [181]:
def get_repair_slices(veh_ids, snapshots, repairs):
    repair_slices = {}
    for veh_id in veh_ids:
        v_snapshots = snapshots[snapshots['Veh Ref ID'] == veh_id].sort_values(by='Event DateTime')
        v_repairs = repairs[repairs['Chassis\nReference\nNumber'] == veh_id].sort_values(by='Rpr_Dt')

        start_date = '1/1/2000' ## in past so first snapshot is captured
        end_date = '1/1/2020' ## in future so last snapshot is captured

        repair_slices[veh_id] = {}
        veh_slices = repair_slices[veh_id]

        ## Best indicator of repair type is the ATA9 code
        ## Iterate over each repair type and append slices
        for repair_type, repair_group in veh_repairs.groupby(['ATA9']):
            i = 0
            start = start_date
            end = -1

            ## dates of all of the repairs in that group
            repair_dates = repair_group.sort_values(by='Rpr_Dt')['Rpr_Dt']
            r_size = len(repair_dates)

            veh_slices_repair = []
            for repair in repair_group.itertuples():
                end = repair[7] ##['Rpr_Dt']

                ## grab a slice of snapshots from end of last repair to start of current repair
                mask = (veh_snapshots['Event DateTime'] >= start) & (veh_snapshots['Event DateTime'] < end)
                snapshot_slice = v_snapshots.loc[mask]

                if len(snapshot_slice) > 0:
                    veh_slices_repair.append(snapshot_slice)

                ## reset start to end for next iteration
                start = end
                i+=1
            
            if len(veh_slices_repair) > 0:
                veh_slices[repair_type] = veh_slices_repair

    return repair_slices

In [182]:
repair_slices = get_repair_slices([797], veh_snapshots, veh_repairs)

In [183]:
repair_slices[797].keys()

dict_keys([101001001, 45021003, 43006007])

In [184]:
len(repair_slices[797][43006007])

1

In [186]:
len(repair_slices[797][45021003][0])

160

In [191]:
veh_snapshots = selected_w[selected_w['Veh Ref ID'].isin(veh_ids)]
veh_repairs = selected_repairs[selected_repairs['Chassis\nReference\nNumber'].isin(veh_ids)]

In [192]:
repair_slices_all = get_repair_slices(veh_ids, veh_snapshots, veh_repairs)

In [193]:
len(repair_slices_all)

13

In [194]:
repair_slices_all.keys()

dict_keys([801.0, 25.0, 616.0, 340.0, 555.0, 751.0, 40.0, 180.0, 373.0, 249.0, 724.0, 495.0, 27.0])

In [195]:
repair_slices_all[801].keys()

dict_keys([42004001, 45021003, 44004001, 45002072, 45007001, 43006007])

In [197]:
len(repair_slices_all[801][45021003][0])

142